In [ ]:
import torch
import torch.nn as nn
import quartz

In [ ]:
t_max = 4

values = torch.tensor([2, 1.5, 1, 0.66, 0.5, 0.4, 0.2354, 0.001, 0,])
values = values.clamp(max=1 + (t_max-1)/t_max)
values

In [ ]:
times = 2*t_max - 1 - (values*t_max).int()
times

In [ ]:
(2*t_max - 1 - times) / t_max# .flip(dims=[0]) 

In [ ]:
t_max = 2**2+1

values = [0, 0.001, 0.5, 0.77, 1, 2]

for value in values:
    weight = 1

    values = torch.ones((1, 1)) * value
    q_values = quartz.quantize_inputs(values, t_max)

    linear_layer = nn.Linear(1, 1, bias=False)
    linear_layer.weight = torch.nn.Parameter(
        torch.ones_like(linear_layer.weight) * weight
    )
    ann_output = linear_layer(q_values)
    q_ann_output = quartz.quantize_inputs(ann_output, t_max)

    temp_q_values = quartz.encode_inputs(q_values, t_max=t_max)
    print(temp_q_values.ravel())
    temp_linear = linear_layer(temp_q_values.flatten(0, 1)).unflatten(0, (1, -1))
    quartz_layer = quartz.IF(t_max=t_max, rectification=False, record_v_mem=True)
    quartz_output = quartz_layer(temp_linear)
    q_quartz_output = quartz.decode_outputs(quartz_output, t_max=t_max)
    print(quartz_layer.v_mem_recorded.ravel().detach())
    print(quartz_output.ravel().detach())
    print(q_quartz_output)

In [ ]:
quartz_output.flip(dims=[1]) / 2*t_max